### Hyperparameter Tuning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow

In [ ]:
!pip install scikeras

In [ ]:
!pip install --upgrade scikeras scikit-learn

In [2]:
## Loading Necessary Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Bidirectional, Dropout, Input, Layer
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import regularizers
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
import tensorflow.keras.backend as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [3]:
# Load rolling window sequences (shape: [num_samples, window_size, num_features])
X = np.load("C:/Users/SRIJAN SASMAL/Desktop/Infosys Springboard Intern/prognosAI-Infosys-intern-project/milestone_3/week_5/Day_20/rolling_window_sequences.npy")      # Feature array

# Load metadata that contains engine_id, cycle, RUL, etc.
metadata = pd.read_csv("C:/Users/SRIJAN SASMAL/Desktop/Infosys Springboard Intern/prognosAI-Infosys-intern-project/milestone_3/week_5/Day_20/sequence_metadata_with_RUL.csv")
y = metadata["RUL"].values                     # Target RUL array

# Print shapes to verify
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (152559, 30, 66)
y shape: (152559,)


In [4]:
# Split into Training and Validation Sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("Train shape:", X_train.shape, y_train.shape)
print("Val shape:", X_val.shape, y_val.shape)

# Inspect sample batches
print("Sample X_train[0] batch shape:", X_train[0].shape)
print("First 5 RUL targets:", y_train[:5])

Train shape: (122047, 30, 66) (122047,)
Val shape: (30512, 30, 66) (30512,)
Sample X_train[0] batch shape: (30, 66)
First 5 RUL targets: [ 56. 296.  86. 122. 121.]


In [5]:
# Data Pipeline / Generator (optional, for very large data)
# Keras can accept Numpy arrays directly in fit(), so for most practical cases, we can skip writing a custom generator
# If memory is a concern, use tf.data.Dataset.from_tensor_slices
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64).prefetch(1)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64).prefetch(1)

In [6]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5, verbose=1)
model_ckpt = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

In [7]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

# Define your create_model function (unchanged)
def create_model(lstm_units=64, dropout_rate=0.3, learning_rate=0.001):
    model = Sequential([
        Bidirectional(LSTM(lstm_units, activation='tanh', return_sequences=False),
                      input_shape=(X.shape[1], X.shape[2])),
        Dropout(dropout_rate),
        Dense(1)
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Wrap the Keras model with scikeras wrapper without setting params yet
model = KerasRegressor(model=create_model, verbose=0)

# Use 'model__' prefix for model hyperparameters in param_grid
param_grid = {
    'model__lstm_units': [32, 64],
    'model__dropout_rate': [0.2, 0.3],
    'model__learning_rate': [0.001, 0.0001],
    'batch_size': [32],
    'epochs': [30],
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

grid_result = grid.fit(
    X_train, y_train,
    callbacks=[early_stopping, reduce_lr, model_ckpt],
    validation_split=0.2,
)


C:\Users\SRIJAN SASMAL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\SRIJAN SASMAL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\SRIJAN SASMAL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\bidirectional.py:107: User

In [11]:
# Best params and score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.800461 using {'batch_size': 32, 'epochs': 30, 'model__dropout_rate': 0.2, 'model__learning_rate': 0.001, 'model__lstm_units': 64}


In [12]:
# Extract best params
best_params = grid_result.best_params_

# Build model with best params
best_model = create_model(lstm_units=best_params['lstm_units'],
                          dropout_rate=best_params['dropout_rate'],
                          learning_rate=best_params['learning_rate'])

# Train and get history
history = best_model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'],
                         validation_split=0.2, callbacks=[early_stopping, reduce_lr, model_ckpt])

# Plot the curves
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.show()

KeyError: 'lstm_units'